# Custom code for generating response functions & datasets:
- Currently, response functions are multi-dimensional sigmoids meaning all input-output relationships will be monotonic. Eventually, might be nice to support non-monotonic relationships as well, so that certain input features can have an "optimum" with worse performance on either side of the optimum.
- Also note: this currently only works for generating non-formulations datasets. Eventually, want to support formulations as well.

In [1]:
import numpy as np
import pandas as pd
from typing import List, Tuple, Optional

## These functions are doing most of the work:

### Convert ingredient recipe data tables from "Wide" to "Compact" format:

In [2]:
def wide_to_compact_format(df):
    """
    Convert formulation data from wide format to compact format.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in wide format where:
        - Each row is a formulation
        - Each column is an ingredient with its weight percentage
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in compact format with columns:
        - component-1_identifier, component-1_amount, component-2_identifier, component-2_amount, etc.
    """
    # Create an empty list to store the transformed rows
    compact_rows = []
    
    # Iterate through each formulation (row)
    for idx, row in df.iterrows():
        # Get non-zero ingredients and their percentages
        ingredients = row[row > 0]
        
        # Create a new row with alternating ingredient names and percentages
        new_row = {}
        for i, (ingredient_name, percentage) in enumerate(ingredients.items(), 1):
            new_row[f'component-{0+i}_identifier'] = ingredient_name
            new_row[f'component-{0+i}_amount'] = percentage
            
        compact_rows.append(new_row)
    
    # Convert to DataFrame
    result_df = pd.DataFrame(compact_rows)
    
    return result_df

### Convert ingredient recipe data tables from "Compact" to "Wide" format:

In [3]:
def compact_to_wide_format(df):
    """
    Convert formulation data from compact format to wide format.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame in compact format where:
        - Each row is a formulation
        - Columns alternate between ingredient names and weight percentages
    
    Returns:
    pandas.DataFrame: Transformed DataFrame in wide format where:
        - Each row is a formulation
        - Each column is an ingredient with its weight percentage
    """
    # Create a list to store the transformed rows
    wide_rows = []
    
    # Get all unique ingredients across all formulations
    ingredient_columns = [col for col in df.columns if 'Name' in col]
    all_ingredients = set()
    for col in ingredient_columns:
        all_ingredients.update(df[col].dropna().unique())
    
    # Process each formulation
    for idx, row in df.iterrows():
        # Create a dictionary with all ingredients initialized to 0
        formulation = {ingredient: 0 for ingredient in all_ingredients}
        
        # Fill in the actual values
        for i in range(1, len(df.columns) // 2 + 1):
            name_col = f'component-{0+i}_identifier'
            weight_col = f'component-{0+i}_amount'
            
            if name_col in df.columns and pd.notna(row[name_col]):
                ingredient_name = row[name_col]
                formulation[ingredient_name] = row[weight_col]
        
        wide_rows.append(formulation)
    
    # Convert to DataFrame
    result_df = pd.DataFrame(wide_rows)
    
    # Sort columns alphabetically for consistency
    result_df = result_df.reindex(sorted(result_df.columns), axis=1)
    
    return result_df

### Constrained Simplex Sampling

#### TODO: make this a little smarter; currently this is very bad at sampling from small constraint ranges

In [4]:
def sample_from_constrained_simplex(
    n_dimensions: int,
    constraints: Optional[List[Tuple[float, float]]] = None,
    max_attempts: int = 1000
):
    """
    Generate a random point from an N-dimensional simplex with optional element-wise constraints.
    
    Parameters:
        n_dimensions (int): Number of dimensions for the simplex
        constraints (List[Tuple[float, float]], optional): List of (min, max) constraints for each dimension.
            Use None for unconstrained dimensions. Example: [(0.2, 0.4), None, (0, 0.5)]
        max_attempts (int): Maximum number of attempts to find a valid solution
        
    Returns:
        numpy.ndarray: Array of N numbers between 0 and 1 that sum to 1 and satisfy constraints
        
    Raises:
        ValueError: If constraints are impossible to satisfy or if max_attempts is reached
    """

    if n_dimensions==0:
        sample = np.array([])
        return sample

    # Initialize constraints if not provided
    if constraints is None:
        constraints = [None] * n_dimensions
    elif len(constraints) != n_dimensions:
        raise ValueError("Length of constraints must match n_dimensions")
    
    # Validate constraints
    total_min = sum(c[0] for c in constraints if c is not None)
    if total_min > 1:
        raise ValueError("Sum of minimum constraints exceeds 1")
    
    for attempt in range(max_attempts):
        try:
            # Generate initial random sample
            sample = np.random.random(n_dimensions)
            sample = sample / np.sum(sample)  # Normalize to sum to 1
            
            # Apply constraints iteratively
            for _ in range(n_dimensions * 2):  # Allow multiple passes for adjustment
                modified = False
                
                # Adjust values to meet constraints
                for i, constraint in enumerate(constraints):
                    if constraint is not None:
                        min_val, max_val = constraint
                        if sample[i] < min_val:
                            deficit = min_val - sample[i]
                            # Take deficit proportionally from unconstrained elements
                            free_indices = [j for j, c in enumerate(constraints) 
                                         if c is None or (j != i and sample[j] > c[0])]
                            if not free_indices:
                                raise ValueError("Cannot satisfy minimum constraint")
                            weights = np.array([sample[j] for j in free_indices])
                            weights = weights / weights.sum()
                            for j, w in zip(free_indices, weights):
                                sample[j] -= deficit * w
                            sample[i] = min_val
                            modified = True
                        elif sample[i] > max_val:
                            excess = sample[i] - max_val
                            # Distribute excess proportionally to unconstrained elements
                            free_indices = [j for j, c in enumerate(constraints) 
                                         if c is None or (j != i and sample[j] < c[1])]
                            if not free_indices:
                                raise ValueError("Cannot satisfy maximum constraint")
                            sample[free_indices] += excess / len(free_indices)
                            sample[i] = max_val
                            modified = True
                
                # Normalize to sum to 1
                sample = sample / np.sum(sample)
                
                # Check if all constraints are satisfied
                constraints_satisfied = all(
                    c is None or (c[0] <= v <= c[1])
                    for c, v in zip(constraints, sample)
                )
                
                if constraints_satisfied and abs(sum(sample) - 1.0) < 1e-10:
                    return sample
                
                if not modified:
                    break
                    
        except ValueError:
            continue
            
    raise ValueError(f"Could not find valid solution after {max_attempts} attempts")

# Main function

### TODO: allow user to add noise to the response functions (make use of the `noise` argument which currently does nothing)

In [76]:
### D-dimensional sigmoid function with the given set of D coefficients:
def sigmoid(input_row, coefs):
    value = 1 / (1 + np.exp(-1 * np.matmul(input_row, coefs)))
    return value


def build_sythetic_demo_dataset(inputs=5, outputs=1, num_rows=10, noise=0, coefs=None, output_format="compact"):

    ### TODO: allow user to add noise to the response functions (using the `noise` argument)
    
    if isinstance(inputs, int):
        num_inputs = inputs
    else:
        general_inputs = inputs["general"]
        formulation_inputs = inputs["formulation"]
        num_general_inputs = len(general_inputs)
        num_formulation_inputs = len(formulation_inputs)
        all_inputs = list(general_inputs) + list(formulation_inputs)
        num_inputs = len(all_inputs)
        if inputs["formulation"]:
            formulation_constraints = [(formulation_inputs[input_]["min"], formulation_inputs[input_]["max"]) for input_ in formulation_inputs]


    if isinstance(outputs, int):
        num_outputs = outputs
    else:
        num_outputs = len(outputs)  


    # Randomly set coefficients for the response function, if not set by the user   
    if coefs==None:
        coefs = np.array([[np.random.uniform(-1, 1) for i in range(num_inputs)] for k in range(num_outputs)])


    # Create pandas DataFrame for the response function coefficients & name the columns
    coefs_df = pd.DataFrame(coefs)
    if isinstance(inputs, int):
        coefs_df = coefs_df.rename(columns={i: f"x_{i+1}" for i in range(len(coefs_df.T))})
        coefs_df = coefs_df.rename(index={k: f"y_{k+1}" for k in range(len(coefs_df))})
    else:
        coefs_df = coefs_df.rename(columns={i: list(all_inputs)[i] for i in range(len(coefs_df.T))})
        coefs_df = coefs_df.rename(index={k: list(outputs)[k] for k in range(len(coefs_df))})

    
    # Generate input values
    if isinstance(inputs, int):
        num_inputs = inputs
        X = np.array([[np.random.uniform(-2, 2) for i in range(num_inputs)] for j in range(num_rows)])
    else:
        X_general = np.array([[np.random.uniform(-2, 2) for i in range(num_general_inputs)] for j in range(num_rows)])
        if inputs["formulation"]:
            
            
            
            
            # X_formulation = np.array([sample_from_constrained_simplex(n_dimensions=num_formulation_inputs, constraints=formulation_constraints) for j in range(num_rows)])
            X_formulation = gibbs_sample_formulation(n_ingredients=num_formulation_inputs, constraints=formulation_constraints, n_samples=num_rows)
            


            X = np.concatenate((X_general, X_formulation), axis=1)
        else:
            X = X_general


    # Generate output values
    y = list()
    for k in range(num_outputs):
        y.append(list())
        for row in X:
            y[k].append(sigmoid(row, coefs[k]))

    y = np.array(y)

    
    # Create pandas DataFrame for the generated data & name the columns
    data_df = pd.DataFrame()

    for i in range(num_inputs):
        if isinstance(inputs, int):
            data_df[f"x_{i+1}"] = X[:, i]
        else:
            data_df[all_inputs[i]] = X[:, i]
    
    for k in range(num_outputs):
        if isinstance(outputs, int):
            data_df[f"y_{k+1}"] = y[k]
        else:
            data_df[list(outputs)[k]] = y[k]


    ### TODO: clean this section up
    #################################
    if isinstance(inputs, int):
        pass
    else:
        df_scaled = data_df.copy()

        for col in df_scaled.columns:
            if col in general_inputs:
                scaled_col = (df_scaled[col].to_numpy() + 2) / 4
            else:
                scaled_col = df_scaled[col]
            df_scaled[col] = scaled_col

        all_columns = dict()
        # all_columns.update(all_inputs)
        all_columns.update(general_inputs)
        all_columns.update(formulation_inputs)
        all_columns.update(outputs)

        for col in all_columns:
            if col in general_inputs or col in outputs:
                df_scaled[col] = df_scaled[col] * (all_columns[col]["max"] - all_columns[col]["min"]) + all_columns[col]["min"]

        column_renaming = {col: f'{col}-{all_columns[col]["units"]}' for col in general_inputs or col in outputs}
        df_scaled = df_scaled.rename(column_renaming, axis=1)
        coefs_df = coefs_df.rename(column_renaming, axis=0)
        coefs_df = coefs_df.rename(column_renaming, axis=1)

        data_df = df_scaled

        if output_format == "compact":
            formulation_column_headers = list(formulation_inputs.keys())
            formulation_df = data_df[formulation_column_headers] * 100
            formulation_df = wide_to_compact_format(formulation_df)
            data_df = data_df.drop(labels=formulation_column_headers, axis=1)
            data_df = pd.concat([data_df, formulation_df], axis=1)
        elif output_format == "wide":
            pass
        else:
            raise ValueError("argument `output_format` must be either 'compact' or 'wide'.")
    #################################
    
    return data_df, coefs_df

## Examples

### Example 1: generate arbitrary # of rows & columns, with no column names

In [6]:
data_df, coefs_df = build_sythetic_demo_dataset(inputs=9, outputs=4, num_rows=10)
data_df

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9,y_1,y_2,y_3,y_4
0,-1.831901,0.374474,0.363485,-0.051258,-1.643307,1.144970,-1.972104,-0.643643,1.691725,0.700688,0.156648,0.837035,0.155340
1,-0.853878,0.986207,-0.970523,0.574263,0.542782,-1.022101,1.404627,-0.258983,0.795018,0.978079,0.913971,0.832590,0.506661
2,-1.086507,-1.275222,0.401101,-0.692694,0.105056,1.271061,1.973360,-1.518899,-0.513738,0.038959,0.773433,0.016749,0.801314
3,0.142940,-0.885628,-1.284463,1.342422,1.585371,-1.460395,-1.679981,-0.927799,-1.958484,0.580477,0.933744,0.967630,0.101060
4,0.447206,0.606576,-0.090172,0.919875,0.114111,1.086188,1.831340,1.180786,1.989101,0.923727,0.124529,0.366782,0.947193
5,0.531769,0.294943,-0.346648,1.878131,0.990687,-1.182552,1.496668,1.058632,-1.778489,0.806368,0.813851,0.671600,0.899195
6,-0.511760,1.909407,0.231146,0.184044,1.457000,-0.106261,0.364931,-0.892399,-0.678804,0.838618,0.982434,0.891557,0.802246
7,-0.940068,-0.414297,-1.434238,1.166609,-1.595431,0.727200,0.532970,1.588565,-1.269120,0.694294,0.386670,0.487959,0.393019
8,0.608455,-1.516428,-0.048395,0.093411,0.172220,0.724355,-0.056866,1.698182,1.094104,0.477679,0.029695,0.242043,0.252550
9,-1.443572,1.006865,0.683569,0.665940,-0.317949,1.872421,0.707357,0.945248,-0.694969,0.618910,0.639664,0.464353,0.865962


In [7]:
coefs_df

,x_1,x_2,x_3,x_4,x_5,x_6,x_7,x_8,x_9
y_1,-0.666803,0.921531,-0.844328,0.488802,0.486248,-0.537316,-0.044878,0.624833,0.794370
y_2,-0.654689,0.847920,-0.716736,-0.439967,0.816008,-0.187541,0.246998,-0.563763,-0.758005
y_3,-0.368988,0.898877,-0.685251,0.619316,0.567103,-0.332928,-0.992575,0.129489,0.203106
y_4,0.950383,0.576093,0.812225,0.986242,-0.332057,0.616652,0.905260,-0.595282,-0.155060


### Example 2: create a laser welding dataset with named columns

#### Assign "reasonable" ranges and desired units for each input & output column:

In [8]:
inputs = {
    "general": {
        "Laser Power": {"min": 100, "max": 1000, "units": "W"},
        "Pulse Duration": {"min": 0.1, "max": 10, "units": "ms"},
        "Welding Speed": {"min": 1, "max": 200, "units": "mm/s"},
        "Beam Diameter": {"min": 0.1, "max": 3, "units": "mm"},
        "Focal Position": {"min": -2, "max": 5, "units": "mm"},
        # "Shielding Gas Type": {"min": , "max": , "units": "n/a"},  # leave out categorical inputs for now
        "Flow Rate": {"min": 5, "max": 25, "units": "L/min"},
        "Heat Input": {"min": 10, "max": 500, "units": "J/mm"},
        "Ambient Temperature": {"min": 20, "max": 30, "units": "degC"},
        "Cooling Rate": {"min": 10, "max": 1000, "units": "degC/s"},
    },
    "formulation": {
        "Carbon": {"min": 0.0, "max": 0.0008, "units": "%"},
        "Manganese": {"min": 0.00, "max": 0.02, "units": "%"},
        "Molybdenum": {"min": 0.01, "max": 0.05, "units": "%"},
        "Nickel": {"min": 0.05, "max": 0.70, "units": "%"},
        "Chromium": {"min": 0.10, "max": 0.40, "units": "%"},
        "Iron": {"min": 0.0, "max": 1.0, "units": "%"},
        "Gold": {"min": 0.0, "max": 1.0, "units": "%"}
    },
}

outputs = {
    "Hardness": {"min": 200, "max": 800, "units": "HV"},
    "Fatigue Life": {"min": 10000, "max": 100000, "units": "numCycles"},
    "Wear Rate": {"min": 0.01, "max": 1.0, "units": "mg/m"},
    "Cutting Efficiency": {"min": 0.1, "max": 5, "units": "m/s"},
}

In [9]:
data_df, coefs_df = build_sythetic_demo_dataset(inputs=inputs, outputs=outputs, num_rows=30)
data_df

,Laser Power-W,Pulse Duration-ms,Welding Speed-mm/s,Beam Diameter-mm,Focal Position-mm,Flow Rate-L/min,Heat Input-J/mm,Ambient Temperature-degC,Cooling Rate-degC/s,Hardness,...,component-3_identifier,component-3_amount,component-4_identifier,component-4_amount,component-5_identifier,component-5_amount,component-6_identifier,component-6_amount,component-7_identifier,component-7_amount
0,636.831718,4.563277,199.767592,1.487588,2.879291,9.546278,224.854014,27.291432,51.208484,375.824057,...,Molybdenum,5.0,Nickel,29.021801,Chromium,13.683619,Iron,17.036839,Gold,33.177742
1,602.988304,8.891607,20.090763,1.776043,2.907412,17.559358,311.996845,29.338071,156.258511,643.686849,...,Molybdenum,5.0,Nickel,14.851086,Chromium,28.645862,Iron,18.394950,Gold,31.028102
2,641.118421,9.004249,187.982070,0.913338,1.975669,18.357091,212.502235,25.931897,879.769957,746.008549,...,Molybdenum,5.0,Nickel,23.370645,Chromium,19.605672,Iron,26.296443,Gold,23.647240
3,865.263191,9.866294,103.998173,0.199949,3.137152,20.479761,61.244216,29.655528,379.731096,574.417201,...,Molybdenum,5.0,Nickel,37.750078,Chromium,11.620802,Iron,22.077303,Gold,21.471816
4,410.648040,7.159512,62.565873,0.108038,2.535477,8.469790,102.104086,25.005526,617.923275,649.176829,...,Molybdenum,5.0,Nickel,19.355071,Chromium,21.424731,Iron,15.824639,Gold,36.315559
5,108.472410,2.084778,157.663582,1.432468,3.215661,14.400614,235.224792,28.256540,218.103605,218.995853,...,Molybdenum,5.0,Nickel,28.155856,Chromium,12.413737,Iron,28.126247,Gold,24.224161
6,992.046529,7.171738,193.900490,1.894855,2.046858,15.639113,452.147707,24.609207,851.266995,796.078490,...,Molybdenum,5.0,Nickel,28.791173,Chromium,28.959100,Iron,29.818268,Gold,5.351459
7,764.587691,6.439753,15.172638,1.697172,2.500283,10.364977,348.818037,24.390897,485.101940,780.983189,...,Molybdenum,5.0,Nickel,22.789810,Chromium,18.974392,Iron,31.490701,Gold,19.665097
8,829.524782,2.005972,183.300187,2.977214,3.167808,7.253466,182.067702,29.138670,721.632883,693.022876,...,Molybdenum,5.0,Nickel,27.217139,Chromium,23.876156,Iron,19.847426,Gold,21.979279
9,849.087304,9.312906,152.590796,1.485994,3.515948,15.370796,267.799473,26.315804,865.818393,790.891970,...,Molybdenum,5.0,Nickel,30.108513,Chromium,24.262510,Iron,15.777105,Gold,22.771872


In [10]:
coefs_df

,Laser Power-W,Pulse Duration-ms,Welding Speed-mm/s,Beam Diameter-mm,Focal Position-mm,Flow Rate-L/min,Heat Input-J/mm,Ambient Temperature-degC,Cooling Rate-degC/s,Carbon,Manganese,Molybdenum,Nickel,Chromium,Iron,Gold
Hardness,0.731718,0.937841,-0.197395,0.100812,0.065676,-0.778550,0.788969,-0.092052,0.852634,-0.019188,0.926416,0.131919,0.813772,0.913552,0.688808,-0.418454
Fatigue Life,0.293277,-0.318035,0.668309,0.927327,0.134315,-0.572118,-0.158267,0.462200,0.414926,0.386551,-0.683445,0.427137,-0.967927,-0.336532,-0.659470,0.777577
Wear Rate,-0.141215,-0.993870,0.212246,-0.512466,-0.742997,-0.244089,0.444111,-0.255782,0.781192,0.109665,-0.864696,-0.609710,0.396376,0.588872,-0.885131,-0.009030
Cutting Efficiency,0.973277,0.075147,0.628478,0.203997,0.962184,-0.071472,-0.170964,-0.279409,-0.671560,0.005112,0.854321,-0.863078,-0.860929,-0.205613,0.772516,0.355271


## [Optional] Save result to Excel or CSV file: 

### Convert ingredient recipe data tables from "Wide" to "Compact" format:

In [11]:
# data_df.to_excel("Demo Datasets/Laser Welding (Synthetic)/laser_welding.xlsx", index=False)
# data_df.to_csv("Demo Datasets/Laser Welding (Synthetic)/laser_welding.csv", index=False)

# data_df.to_csv("./laser_welding_with_formulation.csv", index=False)

# Done!

# Scratch:

## Gibbs Sampling:

In [11]:
import numpy as np

def gibbs_sample_formulation(n_ingredients, mins, maxs, n_samples=1000, burn_in=100):
    """
    Generate samples of ingredient formulations using Gibbs sampling.
    
    Parameters:
    - n_ingredients: number of ingredients
    - mins: minimum percentages (array of length n_ingredients)
    - maxs: maximum percentages (array of length n_ingredients)
    - n_samples: number of samples to generate
    - burn_in: number of initial samples to discard
    
    Returns:
    - samples: array of shape (n_samples, n_ingredients)
    """
    # Validate that a solution is possible
    if sum(mins) > 100 or sum(maxs) < 100:
        raise ValueError("Constraints don't allow for a solution that sums to 100%")
    
    # Initialize with a valid starting point
    current = np.array(mins, dtype=float)
    remaining = 100 - sum(current)
    
    # Distribute remaining amount within constraints
    room_for_more = np.array([max_val - min_val for min_val, max_val in zip(mins, maxs)])
    if sum(room_for_more) > 0:  # Avoid division by zero
        distribution = room_for_more / sum(room_for_more) * remaining
        # Ensure we don't exceed max values
        for i in range(n_ingredients):
            add_amount = min(distribution[i], maxs[i] - current[i])
            current[i] += add_amount
            remaining -= add_amount
    
    # If there's still remaining percentage, distribute it to ingredients that have room
    while remaining > 1e-10:
        can_increase = [i for i in range(n_ingredients) if current[i] < maxs[i]]
        if not can_increase:
            break
        idx = np.random.choice(can_increase)
        add_amount = min(remaining, maxs[idx] - current[idx])
        current[idx] += add_amount
        remaining -= add_amount
    
    # Verify sum is 100%
    assert abs(sum(current) - 100) < 1e-10, f"Initial point doesn't sum to 100%: {sum(current)}"
    
    # Collect samples
    samples = []
    for t in range(n_samples + burn_in):
        # Perform several Gibbs steps per iteration for better mixing
        for _ in range(n_ingredients):
            # Select two random ingredients to adjust
            i, j = np.random.choice(n_ingredients, 2, replace=False)
            
            # Calculate valid range for transfer
            delta_min = max(mins[i] - current[i], current[j] - maxs[j])
            delta_max = min(maxs[i] - current[i], current[j] - mins[j])
            
            if delta_max > delta_min:
                # Sample transfer amount
                delta = np.random.uniform(delta_min, delta_max)
                
                # Update formulation
                current[i] += delta
                current[j] -= delta
        
        # Only keep samples after burn-in
        if t >= burn_in:
            samples.append(current.copy())
    
    return np.array(samples)



In [12]:
# Example usage
n_ingredients = 5
mins = [5, 0, 15, 0, 10]  # Minimum percentages
maxs = [20, 30, 40, 0.15, 100]  # Maximum percentages

samples = gibbs_sample_formulation(n_ingredients, mins, maxs, n_samples=10)


# Verify the results
samples_df = pd.DataFrame(samples)
row_sums = []
for i in range(len(samples_df)):
    row = samples_df.iloc[i]
    row_sum = 0
    for i in row:
        row_sum += i
    row_sums.append(row_sum)

samples_df["Sum"] = row_sums

samples_df

,0,1,2,3,4,Sum
0,14.305920,8.631572,17.678819,0.131406,59.252283,100.0
1,5.280794,6.200015,18.342019,0.057400,70.119773,100.0
2,19.832588,3.411207,33.516925,0.046505,43.192775,100.0
3,5.074129,3.395780,20.050383,0.087760,71.391948,100.0
4,10.856905,3.416940,23.187467,0.106343,62.432344,100.0
5,10.908565,9.638980,18.480001,0.114732,60.857722,100.0
6,17.930471,10.767331,16.092487,0.027119,55.182593,100.0
7,16.295438,8.454922,17.713015,0.097853,57.438772,100.0
8,11.718214,4.859698,19.812142,0.104737,63.505210,100.0
9,6.927082,16.221237,15.845596,0.050000,60.956084,100.0


In [63]:
def gibbs_sample_formulation(
    n_ingredients,
    constraints: Optional[List[Tuple[float, float]]] = None,
    n_samples=100,
    burn_in=100
):
    """
    Generate samples of ingredient formulations using Gibbs sampling.
    
    Parameters:
    - n_ingredients: number of ingredients
    - mins: minimum percentages (array of length n_ingredients)
    - maxs: maximum percentages (array of length n_ingredients)
    - n_samples: number of samples to generate
    - burn_in: number of initial samples to discard
    
    Returns:
    - samples: array of shape (n_samples, n_ingredients)
    """

    if constraints is None:
        constraints = [None] * n_ingredients
    elif len(constraints) != n_ingredients:
        raise ValueError(f"Length of formulation constraints (provided: {len(constraints)}) must equal n_ingredients (provided: {n_ingredients}).")
    

    mins = [c[0] for c in constraints if c is not None]
    maxs = [c[1] for c in constraints if c is not None]
    
    # Validate that a solution is possible
    if sum(mins) > 1:
        raise ValueError("Sum of formulation lower bounds exceeds 1; these constraints don't allow for a solution that sums to 1.")
    elif sum(maxs) < 1:
        raise ValueError("Sum of formulation upper bounds is less than 1; these constraints don't allow for a solution that sums to 1.")
    
    # Initialize with a valid starting point
    current = np.array(mins, dtype=float)
    remaining = 1 - sum(current)
    
    # Distribute remaining amount within constraints
    room_for_more = np.array([max_val - min_val for min_val, max_val in zip(mins, maxs)])
    if sum(room_for_more) > 0:  # Avoid division by zero
        distribution = room_for_more / sum(room_for_more) * remaining
        # Ensure we don't exceed max values
        for i in range(n_ingredients):
            add_amount = min(distribution[i], maxs[i] - current[i])
            current[i] += add_amount
            remaining -= add_amount
    
    # If there's still remaining percentage, distribute it to ingredients that have room
    while remaining > 1e-12:
        can_increase = [i for i in range(n_ingredients) if current[i] < maxs[i]]
        if not can_increase:
            break
        idx = np.random.choice(can_increase)
        add_amount = min(remaining, maxs[idx] - current[idx])
        current[idx] += add_amount
        remaining -= add_amount
    
    # Verify sum is 1 (or 100%)
    ### TODO: is this assertion statement accurate???
    assert abs(sum(current) - 1) < 1e-12, f"Ingredient quantities of initial point don't sum to 1: {sum(current)}" 
    
    # Collect samples
    samples = []
    for t in range(n_samples + burn_in):
        # Perform several Gibbs steps per iteration for better mixing
        for _ in range(n_ingredients):
            # Select two random ingredients to adjust
            i, j = np.random.choice(n_ingredients, 2, replace=False)
            
            # Calculate valid range for transfer
            delta_min = max(mins[i] - current[i], current[j] - maxs[j])
            delta_max = min(maxs[i] - current[i], current[j] - mins[j])
            
            if delta_max > delta_min:
                # Sample transfer amount
                delta = np.random.uniform(delta_min, delta_max)
                
                # Update formulation
                current[i] += delta
                current[j] -= delta
        
        # Only keep samples after burn-in
        if t >= burn_in:
            samples.append(current.copy())
    
    return np.array(samples)

In [64]:
{
    "Carbon": {"min": 0.0, "max": 0.0008, "units": "%"},
    "Manganese": {"min": 0.00, "max": 0.02, "units": "%"},
    "Molybdenum": {"min": 0.01, "max": 0.05, "units": "%"},
    "Nickel": {"min": 0.05, "max": 0.70, "units": "%"},
    "Chromium": {"min": 0.10, "max": 0.40, "units": "%"},
    "Iron": {"min": 0.0, "max": 1.0, "units": "%"},
    "Gold": {"min": 0.0, "max": 1.0, "units": "%"}
}

{'Carbon': {'min': 0.0, 'max': 0.0008, 'units': '%'},
 'Manganese': {'min': 0.0, 'max': 0.02, 'units': '%'},
 'Molybdenum': {'min': 0.01, 'max': 0.05, 'units': '%'},
 'Nickel': {'min': 0.05, 'max': 0.7, 'units': '%'},
 'Chromium': {'min': 0.1, 'max': 0.4, 'units': '%'},
 'Iron': {'min': 0.0, 'max': 1.0, 'units': '%'},
 'Gold': {'min': 0.0, 'max': 1.0, 'units': '%'}}

In [68]:
# Example usage
n_ingredients = 7
# constraints = [(5,20), (0,30), (15,40), (0,0.15), (10,100)]
# constraints = [(.05, .2), (0,.3), (.15,.4), (0.0010,0.0015), (.1,1)]
constraints = [(.00, .0008), (0,0.02), (.01,.05), (0.05,0.70), (.1,0.4), (0,1), (0,1)]
n_samples = 10

samples = gibbs_sample_formulation(n_ingredients, constraints, n_samples=n_samples)


# Verify the results
samples_df = pd.DataFrame(samples)
row_sums = []
for i in range(len(samples_df)):
    row = samples_df.iloc[i]
    row_sum = 0
    for i in row:
        row_sum += i
    row_sums.append(row_sum)

samples_df["Sum"] = row_sums

samples_df

,0,1,2,3,4,5,6,Sum
0,0.000083,0.018593,0.027151,0.094410,0.200367,0.196640,0.462755,1.0
1,0.000130,0.006883,0.030961,0.457620,0.315984,0.187666,0.000755,1.0
2,0.000042,0.018360,0.047876,0.182894,0.186196,0.563876,0.000755,1.0
3,0.000411,0.011608,0.028049,0.215024,0.162509,0.459222,0.123176,1.0
4,0.000435,0.008690,0.046204,0.217933,0.227896,0.334352,0.164488,1.0
5,0.000435,0.004317,0.025221,0.057718,0.254719,0.131229,0.526361,1.0
6,0.000435,0.000052,0.039602,0.088814,0.254719,0.237376,0.379001,1.0
7,0.000738,0.011304,0.039244,0.288215,0.205266,0.076294,0.378938,1.0
8,0.000738,0.013560,0.048987,0.282036,0.207099,0.159964,0.287616,1.0
9,0.000451,0.009674,0.039933,0.282036,0.240007,0.155822,0.272077,1.0


In [69]:
# constraints = [(.05, .2), (0,.3), (.15,.4), (0.0005,0.0015), (.1,1)]
constraints = [(.00, .0008), (0,0.02), (.01,.05), (0.05,0.70), (.1,0.4), (0,1), (0,1)]
n_ingredients = 7
n_samples = 10

samples_df = pd.DataFrame(np.array([sample_from_constrained_simplex(n_dimensions=n_ingredients, constraints=constraints) for j in range(10)]))

row_sums = []
for i in range(len(samples_df)):
    row = samples_df.iloc[i]
    row_sum = 0
    for i in row:
        row_sum += i
    row_sums.append(row_sum)

samples_df["Sum"] = row_sums

samples_df

,0,1,2,3,4,5,6,Sum
0,0.0008,0.02,0.05,0.129094,0.335326,0.218832,0.245948,1.0
1,0.0008,0.02,0.05,0.142520,0.211610,0.268110,0.306960,1.0
2,0.0008,0.02,0.05,0.246816,0.212787,0.207458,0.262139,1.0
3,0.0008,0.02,0.05,0.296051,0.140730,0.278630,0.213788,1.0
4,0.0008,0.02,0.05,0.169032,0.121409,0.348363,0.290396,1.0
5,0.0008,0.02,0.05,0.218545,0.167939,0.276812,0.265904,1.0
6,0.0008,0.02,0.05,0.241905,0.223312,0.140922,0.323061,1.0
7,0.0008,0.02,0.05,0.294359,0.179105,0.235843,0.219893,1.0
8,0.0008,0.02,0.05,0.265402,0.231069,0.170891,0.261839,1.0
9,0.0008,0.02,0.05,0.253651,0.158024,0.249669,0.267856,1.0


In [40]:
gibbs_sample_formulation(n_ingredients, constraints, n_samples=n_samples)

array([[0.18001426, 0.29276679, 0.33843882, 0.00144987, 0.18733026],
       [0.18755197, 0.26580374, 0.33893127, 0.00084334, 0.20686968],
       [0.07129545, 0.16458726, 0.34943249, 0.00084334, 0.41384145],
       [0.14270834, 0.09443541, 0.23750528, 0.00101966, 0.52433131],
       [0.08896049, 0.21196222, 0.38076924, 0.00101966, 0.3172884 ],
       [0.11963772, 0.28391783, 0.26969073, 0.00122276, 0.32553096],
       [0.11361933, 0.2902922 , 0.26974571, 0.00078343, 0.32555932],
       [0.11361933, 0.20796101, 0.30895018, 0.00108165, 0.36838783],
       [0.17156037, 0.20951266, 0.30895018, 0.00071465, 0.30926213],
       [0.13979797, 0.24039788, 0.33625395, 0.0014614 , 0.2820888 ]])

## Try out the code using the new gibbs_sample_formulation function instead of the old sample_from_constrained_simplex:

In [74]:
inputs = {
    "general": {
        "Laser Power": {"min": 100, "max": 1000, "units": "W"},
        "Pulse Duration": {"min": 0.1, "max": 10, "units": "ms"},
        "Welding Speed": {"min": 1, "max": 200, "units": "mm/s"},
        "Beam Diameter": {"min": 0.1, "max": 3, "units": "mm"},
        "Focal Position": {"min": -2, "max": 5, "units": "mm"},
        # "Shielding Gas Type": {"min": , "max": , "units": "n/a"},  # leave out categorical inputs for now
        "Flow Rate": {"min": 5, "max": 25, "units": "L/min"},
        "Heat Input": {"min": 10, "max": 500, "units": "J/mm"},
        "Ambient Temperature": {"min": 20, "max": 30, "units": "degC"},
        "Cooling Rate": {"min": 10, "max": 1000, "units": "degC/s"},
    },
    "formulation": {
        "Carbon": {"min": 0.0, "max": 0.0008, "units": "%"},
        "Manganese": {"min": 0.00, "max": 0.02, "units": "%"},
        "Molybdenum": {"min": 0.01, "max": 0.05, "units": "%"},
        "Nickel": {"min": 0.05, "max": 0.70, "units": "%"},
        "Chromium": {"min": 0.10, "max": 0.40, "units": "%"},
        "Iron": {"min": 0.0, "max": 1.0, "units": "%"},
        "Gold": {"min": 0.0, "max": 1.0, "units": "%"}
    },
}

outputs = {
    "Hardness": {"min": 200, "max": 800, "units": "HV"},
    "Fatigue Life": {"min": 10000, "max": 100000, "units": "numCycles"},
    "Wear Rate": {"min": 0.01, "max": 1.0, "units": "mg/m"},
    "Cutting Efficiency": {"min": 0.1, "max": 5, "units": "m/s"},
}

In [75]:
data_df, coefs_df = build_sythetic_demo_dataset(inputs=inputs, outputs=outputs, num_rows=30)
data_df

,Laser Power-W,Pulse Duration-ms,Welding Speed-mm/s,Beam Diameter-mm,Focal Position-mm,Flow Rate-L/min,Heat Input-J/mm,Ambient Temperature-degC,Cooling Rate-degC/s,Hardness,...,component-3_identifier,component-3_amount,component-4_identifier,component-4_amount,component-5_identifier,component-5_amount,component-6_identifier,component-6_amount,component-7_identifier,component-7_amount
0,593.154651,4.548265,181.579494,1.625851,1.250768,8.185594,163.348598,26.458313,899.552873,527.420648,...,Molybdenum,3.358011,Nickel,25.488558,Chromium,19.483675,Iron,37.278053,Gold,12.749493
1,867.831871,6.403594,98.412451,0.477814,1.247682,19.243378,474.923691,23.190489,964.834324,345.927226,...,Molybdenum,2.834155,Nickel,25.488558,Chromium,19.955474,Iron,46.024032,Gold,4.421638
2,591.375701,2.017243,57.711609,0.656755,1.605074,19.873533,296.647998,29.889895,48.210872,758.014448,...,Molybdenum,4.829224,Nickel,25.529846,Chromium,12.810131,Iron,43.789730,Gold,12.721402
3,833.337618,6.559253,23.082495,0.730937,4.619343,7.759191,252.932753,23.355479,668.104951,357.408829,...,Molybdenum,1.401923,Nickel,27.824102,Chromium,12.810131,Iron,30.778277,Gold,26.509377
4,832.334432,6.010900,43.010682,1.332220,0.627803,19.722331,416.593869,23.294450,498.427102,389.430018,...,Molybdenum,3.039407,Nickel,56.014747,Chromium,12.393259,Iron,1.630273,Gold,26.509377
5,779.262456,2.794575,163.622139,2.493055,0.645394,14.774914,151.258724,27.300255,515.696537,535.216135,...,Molybdenum,4.197748,Nickel,41.798296,Chromium,26.609710,Iron,0.267897,Gold,26.563871
6,301.907852,9.589924,10.945442,0.423481,3.001680,9.432291,470.939395,20.477000,268.764896,597.679881,...,Molybdenum,3.538179,Nickel,5.672321,Chromium,26.567799,Iron,2.208494,Gold,60.659333
7,350.631410,6.909072,199.751105,1.309289,-0.021466,7.130059,22.102142,21.370022,798.953597,715.097508,...,Molybdenum,4.981086,Nickel,19.566334,Chromium,14.532855,Iron,2.208494,Gold,58.155844
8,161.831893,9.613657,74.280167,0.672137,3.489288,12.480468,466.799885,29.139728,629.662062,345.045236,...,Molybdenum,4.587766,Nickel,19.164676,Chromium,27.924358,Iron,2.249004,Gold,44.463489
9,229.922011,3.257499,113.283516,0.453851,-0.592198,19.313477,127.900284,28.250578,434.525695,770.853570,...,Molybdenum,4.823825,Nickel,19.164676,Chromium,29.028247,Iron,4.905572,Gold,40.689560


In [72]:
coefs_df

,Laser Power-W,Pulse Duration-ms,Welding Speed-mm/s,Beam Diameter-mm,Focal Position-mm,Flow Rate-L/min,Heat Input-J/mm,Ambient Temperature-degC,Cooling Rate-degC/s,Carbon,Manganese,Molybdenum,Nickel,Chromium,Iron,Gold
Hardness,-0.920204,0.278786,-0.027100,-0.416908,0.974913,0.372047,-0.681749,-0.222789,-0.520609,0.083633,-0.839724,-0.004042,-0.304440,0.574272,0.305746,-0.155226
Fatigue Life,0.820348,-0.726482,-0.299807,0.102218,0.104212,0.340002,-0.851308,-0.605120,0.050596,-0.906423,0.117250,0.238962,-0.997165,0.885272,0.766282,-0.958310
Wear Rate,-0.081068,-0.870386,-0.653987,0.233765,0.835653,-0.389837,-0.704275,-0.872877,-0.769967,-0.099956,0.444928,0.986240,-0.649218,0.378424,0.190620,0.867109
Cutting Efficiency,0.175082,-0.071108,-0.836034,0.087497,0.263737,-0.864870,0.850531,-0.060259,0.185132,-0.987870,-0.061247,-0.768883,0.858485,0.468954,-0.749207,0.208221
